## Seminar part 1: Fun with Word Embeddings

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2022-07-22 12:51:22--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2022-07-22 12:51:23--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc55601e922409557ef9db692bf6.dl.dropboxusercontent.com/cd/0/get/BpjLT9WilvLVzM06B5J_9f9Sggy87BRLaYPG8kRq9AmPd_c9VhTDG9Sn0ScV5XNJTVKLL2rswrCn-4vveKDF46XUkCGCdFop5OihWY2vZ-AcUXoudegYaq9rS3uikacO_SbYEo8uLMbHsRu6hVeKkeurKedr1gVVmMoj7IDRX_1I0Q/file?dl=1# [following]
--2022-07-22 12:51:23--  https://uc55601e922409557ef9db692bf6.dl.dropboxusercontent.com/cd/0/get/BpjLT9WilvLVzM06B5J_9f9Sggy87BRLaYPG8kRq9AmPd_c9VhTDG9Sn0ScV5XNJTVKLL2rswrC

In [2]:
import numpy as np

data = list(open("./quora.txt"))
data = [i.lower() for i in data]

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [4]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(i) for i in data]

In [5]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [6]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [7]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [8]:
# now you can get word vectors !
model.get_vector('anything')

array([-0.47706893, -1.2979316 ,  1.2344617 , -0.45471588, -0.9532109 ,
        0.7262169 ,  1.7495788 , -3.2990797 ,  1.4090775 , -0.6066704 ,
        0.7492886 , -1.6586889 , -3.8144906 ,  1.2084185 ,  0.08039618,
        0.52421016, -0.97906363,  3.0226126 ,  6.0602627 ,  2.7108693 ,
       -1.7119519 ,  0.48177862, -0.74914104, -0.7723529 , -3.8396013 ,
       -0.20987478,  3.2289255 ,  0.927057  , -0.8759403 ,  0.61258453,
       -0.581663  ,  0.51123154], dtype=float32)

In [9]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9654152989387512),
 ('beans', 0.953215479850769),
 ('honey', 0.9433867335319519),
 ('pasta', 0.9347929954528809),
 ('sauce', 0.9252099990844727),
 ('grass', 0.9239892959594727),
 ('cheese', 0.9213664531707764),
 ('butter', 0.915513277053833),
 ('chocolate', 0.9150205850601196),
 ('fruit', 0.9122623205184937)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [10]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [11]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.538511335849762),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233934879303),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [12]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [13]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(word) for word in words])

In [14]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [15]:
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(n_components=2)
s = StandardScaler()

word_vectors_pca = s.fit_transform(pca.fit_transform(word_vectors))

#transformer = Normalizer().fit(word_vectors_pca) 

#word_vectors_pca = transformer.transform(word_vectors_pca)

# and maybe MORE OF YOUR CODE here :)

In [16]:
word_vectors_pca.std(0)

array([1.0000004 , 0.99999994], dtype=float32)

In [17]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [18]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [19]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1003', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [21]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.
# normalize them as just lke with pca


word_tsne = TSNE(n_iter=250, verbose=100).fit_transform(word_vectors)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.000s...
[t-SNE] Computed neighbors for 1000 samples in 0.093s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.716133
[t-SNE] Computed conditional probabilities in 0.117s
[t-SNE] Iteration 50: error = 68.2606354, gradient norm = 0.3041337 (50 iterations in 14.725s)
[t-SNE] Iteration 100: error = 68.1624756, gradient norm = 0.2977087 (50 iterations in 11.415s)
[t-SNE] Iteration 150: error = 68.7326202, gradient norm = 0.3113123 (50 iterations in 11.690s)
[t-SNE] Iteration 200: error = 68.2436905, gradient norm = 0.3073942 (50 iterations in 12.112s)
[t-SNE] Iteration 250: error = 67.8291931, gradient norm = 0.3127545 (50 iterations in 12.290s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 67.829193
[t-SNE] KL divergence after 251 iterations: 17976931348623157081452742373170435679807056752584499659891747680315726078002853876058955863276687817154045895

In [22]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1112', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [45]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    phrase = phrase.lower()
    phrase = tokenizer.tokenize(phrase)
    
    for word in phrase:
      try:
        vector += model.get_vector(word)
      except:
        pass

    vector /= max(len(phrase), 1e-2)
    
    return vector
        
    

In [46]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [50]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [52]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [54]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE(verbose=1000, n_iter=300).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1001 samples in 0.001s...
[t-SNE] Computed neighbors for 1001 samples in 0.095s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1001
[t-SNE] Computed conditional probabilities for sample 1001 / 1001
[t-SNE] Mean sigma: 0.473747
[t-SNE] Computed conditional probabilities in 0.132s
[t-SNE] Iteration 50: error = 79.4173584, gradient norm = 0.3330930 (50 iterations in 24.721s)
[t-SNE] Iteration 100: error = 80.6296844, gradient norm = 0.3222255 (50 iterations in 12.370s)
[t-SNE] Iteration 150: error = 80.4145279, gradient norm = 0.3139785 (50 iterations in 17.807s)
[t-SNE] Iteration 200: error = 82.5508347, gradient norm = 0.3036210 (50 iterations in 12.625s)
[t-SNE] Iteration 250: error = 80.8811951, gradient norm = 0.3192345 (50 iterations in 13.966s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 80.881195
[t-SNE] Iteration 300: error = 1.8827460, gradient norm = 0.0038825 (50 iterations in 10

In [55]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1232', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [82]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data[0:1000]])

In [58]:
from sklearn.metrics.pairwise import cosine_similarity

In [60]:
sorted([1, 3, 2], reverse=True)

[3, 2, 1]

In [115]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    vector = get_phrase_embedding(query)

    d = np.zeros(data_vectors.shape[0])

    #print(data_vectors[0].shape, vector.shape)
    #print(cosine_similarity(np.expand_dims(data_vectors[0], 0), np.expand_dims(vector, 0)))

    for i in np.arange(d.shape[0]):
      d[i] = cosine_similarity(np.expand_dims(data_vectors[i], 0), np.expand_dims(vector, 0))[0]

    max_indices = np.argsort(-d)
    
    return np.array(data)[max_indices[:k]]

In [116]:
find_nearest(query="How do i enter the matrix?", k=10)

array(['how do i make best use of ieee?\n', 'how do i crack the gre?\n',
       'how do i choose a topic for my blog?\n',
       'how can i read this time?\n',
       "i don't have the energy to do anything anymore. what should i do?\n",
       'how do i burn a dvd so that it will play automatically in a dvd player?\n',
       'what should i do if i forgot the answers to my apple id questions?\n',
       'how do i prepare for the ieo exam?\n',
       'how do i write an oath? what are some tips?\n',
       "how do i find my life's goal?\n"], dtype='<U1170')

In [ ]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

In [117]:
find_nearest(query="How does Trump?", k=10)

array(['what does the 12th amendment mean?\n',
       'who is worse, trump or clinton?\n',
       'does cvs still develop film? why or why not?\n',
       'what does entertainment mean for you?\n',
       'how does hstalks pay the speakers?\n',
       'what is the intermediate value theorem? how do you find it?\n',
       'what type of government does turkey have? how does it compare to the one in brazil?\n',
       'how do you achieve the freezing point of hexane? how does it compare to methane?\n',
       'how can you see exactly how may answers a question on quora has when it says 100+ answers?\n',
       'what do you think about rashtriya swayamsevak sangh?\n'],
      dtype='<U1170')

In [118]:
find_nearest(query="Why don't i ask a question myself?", k=10)

array(["why do i smile even when i don't want to?\n",
       "how do i tell my husband i don't think we're ready to try for a baby yet?\n",
       "how can i plan to become a multi billionaire? i'm 19 now?\n",
       "how do i control my son's anger?\n",
       "what is something that you know that i don't?\n",
       "my parents want me to become a doctor but i'm in love with cricket? what can i do to convince them?\n",
       "why don't i feel anxious in the morning?\n",
       "what if i don't pass the ccna exam?\n",
       "i don't have the energy to do anything anymore. what should i do?\n",
       "how do i find my life's goal?\n"], dtype='<U1170')

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.